In [ ]:
from bastionlab import Identity

# Create `Identity` for Data owner.
data_owner = Identity.create("data_owner")

In [ ]:
!wget 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

In [5]:
import polars as pl

df = pl.read_csv("titanic.csv")

from bastionlab import Connection

connection = Connection("localhost")

from bastionlab.polars.policy import Policy, Aggregation, Log

policy = Policy(safe_zone=Aggregation(min_agg_size=10), unsafe_handling=Log())
rdf = connection.client.polars.send_df(df, policy=policy, sanitized_columns=["Name"])

In [ ]:
rdf.columns

In [6]:
import torch
from typing import List, Optional, Union, Tuple


rds = rdf.to_dataset("Name", "Age", "distilbert-base-uncased")

rds
# _rdf.fetch()

In [7]:
remote_datasets = connection.client.torch.list_remote_datasets()
print(remote_datasets)

In [ ]:
! pip install transformers

In [3]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from bastionlab.torch.utils import MultipleOutputWrapper

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
    torchscript=True,
)
model = MultipleOutputWrapper(model, 0)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [4]:
remote_datasets = connection.client.torch.list_remote_datasets()

GRPCException: Connection to the gRPC server failed: code=StatusCode.UNAVAILABLE message=failed to connect to all addresses

In [ ]:
from bastionlab.torch.optimizer_config import Adam


client = connection.client.torch
remote_learner = client.RemoteLearner(
    model,
    remote_datasets[0],
    max_batch_size=2,
    loss="cross_entropy",
    optimizer=Adam(lr=5e-5),
    model_name="DistilBERT",
)